In [ ]:
import os
import json
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
from compound_keywords import compound_keywords

#### NLTK Text Pre-processing:

In [ ]:
def preprocess_text(text):
    # Define a list of English stopwords
    stop_words = set(stopwords.words('english'))
    # Add specific terms to be removed
    remove_terms = {'introduct', 'literature', 'review', 'figure', 'doi', 'fig', 'table', 'conclusion', 
                    'altimg', 'gif', 'png', 'discussion', 'acknowledgment', 'appendix','http', 'copyright'}
    stop_words.update(remove_terms)

    # Initialize lemmatizer and stemmer
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()

    # Convert to lowercase
    text = text.lower()
    for original, compound in compound_keywords.items():
        text = text.replace(original, compound)
    
    # Tokenize the text into sentences
    sentence_tokens = sent_tokenize(text)
    sentences = []

    # Tokenize each sentence into words and remove stopwords
    for sentence in sentence_tokens:
        words = word_tokenize(sentence)
        filtered_words = []
        for word in words:
            word = lemmatizer.lemmatize(word)
            #word = stemmer.stem(word)
            
            # Remove non-alphabetic characters except underscores and hyphens
            word = re.sub(r'[^\w\s\-]', '', word)
            
            #if word.lower() not in stop_words:
            if word.lower() not in stop_words and not word.isdigit() and 1 < len(word) <= 20:
                filtered_words.append(word)
            
        sentences.append(filtered_words)
    
    return sentences

def process_json_files(directory):
    all_sentences = []

    # Iterate through each JSON file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                
                # Extract the original text from the JSON structure
                text = data['full-text-retrieval-response']['originalText']
                # Process the text to tokenize and remove stopwords
                processed_sentences = preprocess_text(text)
                # Append the processed sentences to the overall list
                all_sentences.extend(processed_sentences)
    
    return all_sentences

In [ ]:
# Define the directory path that contains your JSON files
directory = '/.../downloaded_articles'

# De-bug
#processed_sentences = process_json_files(directory)
#print(processed_sentences)

#### Train and Save Word2Vec Model:

In [ ]:
# Define the Word2Vec model
model = Word2Vec(
    sentences=process_json_files(directory),  # Training data (tokenized sentences)
    vector_size=300,           # Size of the embedding vectors
    window=20,                 # Context window size
    min_count=2,               # Minimum occurrence in vocabulary
    workers=4                  # Number of threads for model training
)

In [ ]:
model_save_path = "word2vec_model.model"
model.save(model_save_path)

#### Import Word2Vec Model:

In [ ]:
model = Word2Vec.load("word2vec_model.model")
print("Model loaded successfully.")

In [ ]:
# Find similar words 
similar_words_fabric = model.wv.most_similar('rl', topn=100)
similar_words_textile = model.wv.most_similar('carbon', topn=20)

print("Words similar to 'mass_transfer':")
for word, similarity in similar_words_fabric:
    print(f"{word}: {similarity}")

print("\nWords similar to 'carbon':")
for word, similarity in similar_words_textile:
    print(f"{word}: {similarity}")

In [ ]:
# Find similarity
similarity_1 = model.wv.similarity('build', 'hous')
print(similarity_1)